In [94]:
%load_ext autoreload
%autoreload 2
import numpy as np
from PIL import Image, ImageOps
import os
from utils.H264 import *
from utils.ycbcr_conv import *
from utils.check import *
from pathlib import Path
from contextlib import redirect_stdout
# create directrot output if not exist
if not os.path.exists("output"):
    os.makedirs("output")

def read_image_into_rgb(path):
    img = Image.open(path)
    img = ImageOps.exif_transpose(img)
    img = img.convert("RGB")
    rgb = np.array(img)
    print(rgb.shape, rgb.dtype)              # (H, W, 3) uint8
    return rgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
# ipynb_root = Path(__file__).parent
input_img_path = Path("img") / "trump.jpg"
output_img_path = Path("output") / "recover_trump.jpg"
rgb_image = read_image_into_rgb(input_img_path)
ycbcr_image = rgb_2_ycbcr(rgb_image=rgb_image)
recover_rgb_image = ycbcr_2_rgb(ycbcr_image=ycbcr_image)
Image.fromarray(recover_rgb_image, mode="RGB").save(output_img_path, quality=90, optimize=True, progressive=True)

(512, 512, 3) uint8


/var/folders/fl/376831yd6nb6lgjqjxshzqw40000gn/T/ipykernel_75664/4245845366.py:7: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(recover_rgb_image, mode="RGB").save(output_img_path, quality=90, optimize=True, progressive=True)


In [96]:
original_image = read_image_into_rgb(input_img_path)
recover_image = read_image_into_rgb(output_img_path)
diff_two_tensor(A=original_image, B=recover_image)

(512, 512, 3) uint8
(512, 512, 3) uint8
convertion diff: 0.5409317016601562


In [97]:
Y_tensor = ycbcr_image[...,0]
print(Y_tensor.shape, Y_tensor.dtype)

(512, 512) uint8


In [98]:
frames, _ = macro_block_partition(Y_tensor=Y_tensor)

(16, 16, 32, 32) (512, 512)
convertion diff: 0.0


In [99]:
with open("rec.log","w") as f:
    with redirect_stdout(f):
        target_frame = frames[8, 7]
        h, w = target_frame.shape
        for i in range(h):
            for j in range(w):
                print(f"{target_frame[i, j]:3d}", end=' ')
            print()
        op_mode = [1, 0, 0, 0]
        QP = 13
        blocks, _ = consume_a_frame(frame=target_frame, op_mode=op_mode, QP=QP)